# Add Segmentation Masks to a TableFromCoco

Segmentation support is currently quite limited in 3LC, but we are working hard to improve it. In the meantime, you can use this notebook to add segmentation masks to a TableFromCoco object. This will allow you to use the segmentation masks in your 3LC project.

NOTE: When segmentation polygons are converted to a single-channel PNG, instance information is lost, and overlapping regions will be merged, with the highest value label taking precedence. 

NOTE: Using the masks in training is not yet supported, but might be possible with some extra work.

NOTE: It is possible to edit the masks in the 3LC Dashboard, but the original polygons, or any associated bounding boxes will not be edited.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tlc
from pycocotools.coco import COCO
import pycocotools.mask as mask_util
from PIL import Image
import numpy as np

from tools import add_columns_to_table
from tools.common import data_root


Load a TableFromCoco object and its associated annotations.json file.

In [3]:
# REPLACE THIS WITH YOUR OWN DATA

annotations_file = data_root() / "coco128" / "annotations.json"
image_folder = data_root() / "coco128" / "images"

# annotations_file = data_root() / "balloons/train/train-annotations.json"
# image_folder = data_root() / "balloons/train"

In [4]:
# REPLACE THIS WITH YOUR OWN TABLE

table = tlc.Table.from_coco(
    annotations_file=annotations_file,
    image_folder=image_folder,
    table_name="initial",
    dataset_name="COCO128",
    project_name="3LC Demos",
)


In [ ]:
table

In [ ]:
# The segmentation _is_ in the Table, but connected components have been flattened, and label information has been lost.
# We will fix this in a later release.

table[0]["bbs"]["segmentation"]

Unfortunately, the "segmentation" field in the TableFromCoco is useless, as it does not contain label information.

So, we will need to parse the original annotations.json file, construct new segmentation (polygon) values, create a new column, and add it to the table. Later we can convert the segmentations to masks (png).

 We will construct a list of values for the new "polygon_segments" column of the format:

```python
 {
     "polygons": list[list[list[float]]],
     "labels": list[int],
 }
```

"polygons" dimensions: (n_annotations, n_connected_components, n_points_per_component x 2)


In [ ]:
coco = COCO(annotations_file)
image_ids = sorted(coco.getImgIds())

In [8]:
assert len(image_ids) == len(table)

In [9]:
# Collect segmentation polygons and labels
segmentation_polygons = []

for image_id in image_ids:
    annotation_ids = coco.getAnnIds(imgIds=image_id)
    annotations = coco.loadAnns(annotation_ids)

    polygons = []
    labels = []

    for annotation in annotations:
        segmentation = [[float(x) for x in l] for l in annotation["segmentation"]]
        polygons.append(segmentation)
        labels.append(annotation["category_id"])

    segmentation_polygons.append({"polygons": polygons, "labels": labels})

In [ ]:
segmentation_polygons[0]

In [11]:
# Define a schema for the new column we will add to the table

schema = tlc.Schema("polygon_segments")
value_map = table.get_value_map("bbs.bb_list.label")
assert value_map is not None

segmentation_list_schema = tlc.Schema(
    value=tlc.Float32Value(), 
    size0=tlc.DimensionNumericValue(0, 1000),  # Annotations
    size1=tlc.DimensionNumericValue(0, 100),   # Connected components
    size2=tlc.DimensionNumericValue(0, 10000), # Points per component
)

label_schema = tlc.Schema(
    value=tlc.Int32Value(value_map=value_map),
    size0=tlc.DimensionNumericValue(0, 1000),  # Annotations
)

schema.add_sub_schema("polygons", segmentation_list_schema)
schema.add_sub_schema("labels", label_schema)

column_schemas = {
    "polygon_segments": schema
}

In [12]:
new_table = add_columns_to_table(
    table, 
    {"polygon_segments": segmentation_polygons}, 
    column_schemas, 
    "added-polygon-segments",
    "Added polygon segment column to table",
)

In [13]:
bb_list = new_table[0]["bbs"]["bb_list"]
polygons = new_table[0]["polygon_segments"]["polygons"]
labels = new_table[0]["polygon_segments"]["labels"]

In [ ]:
new_table.columns

## Part 2: Convert Polygon to Mask (PNG)



In [15]:
def polygons_to_mask(polygons: list[list[float]], height: int, width: int) -> np.ndarray:
    formatted_polygons = []

    for polygon in polygons:
        if isinstance(polygon[0], list):  # Handling nested lists of coordinates
            # Flatten the polygon (convert from [[x1, y1], [x2, y2], ...] to [x1, y1, x2, y2, ...])
            flat_polygon = [coord for point in polygon for coord in point]
            formatted_polygons.append(flat_polygon)
        else:
            formatted_polygons.append(polygon)

    # Convert polygons to COCO-style RLE format
    rle = mask_util.frPyObjects(formatted_polygons, height, width)

    # Decode RLE into a binary mask
    mask = mask_util.decode(rle)[:, :]
    
    return mask

In [16]:
def row_to_mask(row):
    image_height = row["height"]
    image_width = row["width"]

    polygons = row["polygon_segments"]["polygons"]
    labels = row["polygon_segments"]["labels"]
    labels = [label + 1 for label in labels]

    if len(polygons) == 0:
        return Image.new("L", (image_width, image_height))
    
    masks = [polygons_to_mask(polygon, image_height, image_width) for polygon in polygons]

    # Create a mask for each label
    label_masks = [np.zeros_like(mask) for mask in masks]

    for mask, orig_mask, label in zip(label_masks, masks, labels):
        mask[orig_mask == 1] = label

    # Combine the masks into a single image
    combined_mask = np.zeros_like(masks[0])

    for mask in label_masks:
        combined_mask = np.maximum(combined_mask, mask)

    mask_image = Image.fromarray(combined_mask.squeeze().astype(np.uint8))
    return mask_image

In [18]:
masks = [row_to_mask(row) for row in new_table]

In [19]:
# Construct a incremented value map (0 is reserved for the background)

map = {0: tlc.MapElement("background")}
for value, label in value_map.items():
    map[value+1] = label

mask_schema = tlc.SegmentationPILImage("mask", classes=map)

In [20]:
# Add the masks to the table
with tlc.bulk_data_url_context(new_table.bulk_data_url.to_absolute(new_table.url), new_table.url):
    mask_table = add_columns_to_table(
        new_table, {"mask": masks},
        {"mask": mask_schema},
        "added-masks",
        "Added masks to table",
    )

In [ ]:
mask_table.table_rows[0]["mask"]